# Animation of 3D models generated by spateo

In [1]:
import io, warnings, os
from pathlib import Path
import base64
from IPython.display import HTML

import spateo as st
warnings.filterwarnings('ignore')

2024-05-27 02:06:24.655531: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/pc001/anaconda3/envs/BioProjects/lib/python3.8/site-packages/umap/distances.py:1063: NumbaDeprecationWarning:

The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.

/home/pc001/anaconda3/envs/BioProjects/lib/python3.8/site-packages/umap/distances.py:1071: NumbaDeprecationWarning:

The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behav

## Load the data

In [2]:
os.chdir(f"/media/pc001/Yao/Projects/Project_spateo/mouse_heart")
out_image_path = f"animations/drosophila_germband_models"
Path(out_image_path).mkdir(parents=True, exist_ok=True)

In [3]:
cpo = [(41, 1209, 57), (13, 8, -3), (0, 0, 1)]
 
sample_id = "E7_8h"
germ_adata = st.read_h5ad(f"/media/pc001/Yao/Projects/Project_drosophila/Data_v4/migration-hotspot/drosophila_{sample_id}_germ_layer_new/h5ad/{sample_id}_germband_v4.h5ad")
germ_pc, _ = st.tdr.construct_pc(
    adata=germ_adata.copy(),
    spatial_key="3d_align_spatial",
    groupby="anno_tissue_new",
    key_added="tissue",
    colormap={"amnioserosa": "#bdcf32", "CNS": "#ea5545", "hindgut": "#27aeef", "midgut": "#87bc45", "muscle": "#f46a9b", "salivary gland": "#ef9b20",},
)
germ_mesh = st.tdr.read_model(f"/media/pc001/Yao/Projects/Project_drosophila/Data_v4/migration-hotspot/drosophila_{sample_id}_germ_layer_new/mesh_models/Germband_{sample_id}_aligned_mesh_model.vtk")
germ_adata

AnnData object with n_obs × n_vars = 12585 × 8120
    obs: 'area', 'slices', 'anno_cell_type', 'anno_tissue', 'anno_germ_layer', 'actual_stage', 'scc', 'anno_tissue_new', 'V_z', 'anno_germ_layer_new', 'backbone_nodes', 'backbone_scc', 'segments', 'morpho_acceleration', 'morpho_curvature', 'morpho_curl', 'morpho_torsion'
    uns: 'VecFld_morpho', 'glm_degs', 'log1p', 'pp'
    obsm: '2d_realign_spatial', '3d_align_spatial', 'V_cells_mapping', 'X_cells_mapping', 'X_pca', 'X_umap', 'morpho_acceleration', 'morpho_curl', 'morpho_curvature', 'morpho_velocity', 'raw_spatial', 'tdr_spatial'
    layers: 'X_counts', 'spliced', 'unspliced'

In [7]:
germ_adata.uns["VecFld_morpho"].keys()

dict_keys(['C', 'E_traj', 'P', 'V', 'VFCIndex', 'X', 'X_ctrl', 'Y', 'beta', 'ctrl_idx', 'grid', 'grid_V', 'iteration', 'method', 'sigma2', 'tecr_traj', 'valid_ind'])

## Animation of mapping model

In [4]:
import numpy as np
stage2_germ_adata = st.read_h5ad(f"/media/pc001/Yao/Projects/Project_drosophila/Data_v4/migration-hotspot/drosophila_E8_9h_germ_layer/h5ad/E8_9h_germ_layer_v3.h5ad")
stage2_germ_pc, _ = st.tdr.construct_pc(
    adata=stage2_germ_adata.copy(),
    spatial_key="3d_align_spatial",
    groupby="anno_tissue_new",
    key_added="tissue",
    colormap={"amnioserosa": "#bdcf32", "CNS": "#ea5545", "hindgut": "#27aeef", "midgut": "#87bc45", "muscle": "#f46a9b", "salivary gland": "#ef9b20",},
)
models_distance = np.asarray([0, 0, -260])
stage2_germ_pc.points = stage2_germ_pc.points + models_distance
stage2_germ_adata

AnnData object with n_obs × n_vars = 11548 × 8451
    obs: 'area', 'slices', 'anno_cell_type', 'anno_tissue', 'anno_germ_layer', 'actual_stage', 'scc', 'anno_tissue_new'
    obsm: '2d_realign_spatial', '3d_align_spatial', 'X_pca', 'X_umap', 'raw_spatial', 'tdr_spatial'
    layers: 'X_counts', 'spliced', 'unspliced'

In [5]:
from dynamo.tools.sampling import trn
sub_arr = trn(X=germ_adata.obsm["3d_align_spatial"], n=5000, return_index=True, seed=19491001)

|-----> [Running TRN] in progress: 100.0000%|-----> [Running TRN] completed [1877.0626s]


In [12]:
model_lines, _ = st.tdr.construct_align_lines(
            model1_points=germ_adata.obsm["3d_align_spatial"][sub_arr].copy(),
            model2_points=germ_adata.obsm["X_cells_mapping"][sub_arr] + models_distance,
            key_added="mapping",
            label="mapping lines",
            color="gainsboro",
            alpha=1.0,
        )

In [24]:
mapping_cpo = [(18, 1370, -141),(-1.8, 5.8, -156.8), (0, 0, 1)]
st.pl.three_d_plot(
    model=st.tdr.collect_models([germ_pc, stage2_germ_pc, model_lines]),
    key=["tissue", "tissue", "mapping"],
    filename=os.path.join(out_image_path, f"Germband_mapping_model_animation.mp4"),
    model_size=[14, 14, 2],
    model_style=["points", "points", "wireframe"],
    opacity=[1, 1, 0.1],
    show_legend=True,
    off_screen=False,
    jupyter="static",
    background="white",
    window_size=(2560, 2048),
    cpo=mapping_cpo,
    text=f"\n3D alignment of Germ band (Drosophila S11&S13)",
    text_kwargs={"font_size":20},
    view_up=(0, 0, 1),
    framerate=6
)

## Animation of trajectory model

In [14]:
st.tdr.morphopath(
    adata=germ_adata,
    # layer="log1p_X",
    vf_key="VecFld_morpho",
    key_added="fate_morpho",
    t_end=2000,
    interpolation_num=50,
    cores=20
)

In [20]:
trajectory_model, _ = st.tdr.construct_trajectory(
    adata=germ_adata,
    fate_key="fate_morpho",
    n_sampling=1500,
    sampling_method="trn",
    key_added="obs_index",
    label=np.asarray(germ_adata.obs.index), # stage1_tissue_adata.uns["VecFld_morpho"]["V"][:, 2].flatten(),
)

germ_adata.obs["V_x"] = germ_adata.uns["VecFld_morpho"]["V"][:, 0].flatten()
st.tdr.add_model_labels(
        model=trajectory_model,
        key_added="V_x",
        labels=np.asarray(germ_adata[np.asarray(trajectory_model.point_data["obs_index"])].obs["V_x"]),
        colormap="Spectral",
        where="point_data",
        inplace=True,
    )

(None, 'Spectral')

In [28]:
cells_models, _ = st.tdr.construct_genesis(
    adata=germ_adata,
    fate_key="fate_morpho",
    n_steps = 100,
    logspace=True,
    t_end=1000,
    label=[germ_adata.uns["VecFld_morpho"]["V"][:, 0]] * 100,
    color="Spectral"
)

In [29]:
st.pl.three_d_animate(
    models=cells_models,
    stable_model=trajectory_model,
    key="genesis",
    stable_kwargs=dict(
        key="V_x",
        model_style="wireframe",
        model_size=5,
        opacity=0.5,
        colormap="Spectral",
        show_legend=False,
    ),
    filename=os.path.join(out_image_path, f"Germband_morphofield_model_animation.mp4"),
    colormap="Spectral",
    model_style="points",
    model_size=14,
    jupyter="static",
    background="white",
    text=f"\nPredicted migration trajectories of Germ band (Drosophila S11)",
    text_kwargs={"font_size":20},
    window_size=(2560, 2048),
    cpo=cpo,
    framerate=6)

## Merge all animations

In [30]:
st.pl.merge_animations(
    mp4_files=[
        os.path.join(out_image_path, f"Germband_mapping_model_animation.mp4"),
        os.path.join(out_image_path, f"Germband_morphofield_model_animation.mp4"),
    ],
    filename=os.path.join(out_image_path, f"Drosophila_Germband_contraction_animation.mp4"),
)

Moviepy - Building video animations/drosophila_germband_models/Drosophila_Germband_contraction_animation.mp4.
Moviepy - Writing video animations/drosophila_germband_models/Drosophila_Germband_contraction_animation.mp4



Moviepy - Done !
Moviepy - video ready animations/drosophila_germband_models/Drosophila_Germband_contraction_animation.mp4
